In [ ]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar


# parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
# parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
# parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
# args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# if args.resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load('./checkpoint/ckpt.pth')
#     net.load_state_dict(checkpoint['net'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..

Epoch: 0
 [================================================================>]  Step: 2s814ms | Tot: 1m32s | Loss: 1.378 | Acc: 49.194% (24597/5000 391/391  
 [================================================================>]  Step: 65ms | Tot: 6s415ms | Loss: 1.161 | Acc: 60.210% (6021/1000 100/100 
Saving..

Epoch: 1
 [================================================================>]  Step: 165ms | Tot: 1m30s | Loss: 0.851 | Acc: 69.732% (34866/5000 391/391 1  
 [================================================================>]  Step: 64ms | Tot: 6s165ms | Loss: 0.886 | Acc: 70.800% (7080/1000 100/100 
Saving..

Epoch: 2
 [================================================================>]  Step: 167ms | Tot: 1m30s | Loss: 0.637 | Acc: 77.856% (38928/5000 391/391 1  
 [================================================================>]  Step: 57ms | Tot: 6s190ms | Lo

 [================================================================>]  Step: 163ms | Tot: 1m30s | Loss: 0.107 | Acc: 96.286% (48143/5000 391/391 1  
 [================================================================>]  Step: 62ms | Tot: 6s187ms | Loss: 0.376 | Acc: 89.100% (8910/1000 100/100 

Epoch: 31
 [================================================================>]  Step: 164ms | Tot: 1m30s | Loss: 0.102 | Acc: 96.454% (48227/5000 391/391 1  
 [================================================================>]  Step: 62ms | Tot: 6s231ms | Loss: 0.313 | Acc: 90.790% (9079/1000 100/100 
Saving..

Epoch: 32
 [================================================================>]  Step: 166ms | Tot: 1m30s | Loss: 0.091 | Acc: 96.938% (48469/5000 391/391 1  
 [================================================================>]  Step: 62ms | Tot: 6s209ms | Loss: 0.340 | Acc: 90.190% (9019/1000 100/100 

Epoch: 33
 [================================================================>]  Step: 159

 [================================================================>]  Step: 58ms | Tot: 6s86ms | Loss: 0.291 | Acc: 91.640% (9164/1000 100/100 
Saving..

Epoch: 60
 [================================================================>]  Step: 161ms | Tot: 1m30s | Loss: 0.067 | Acc: 97.770% (48885/5000 391/391 91 
 [================================================================>]  Step: 62ms | Tot: 5s923ms | Loss: 0.322 | Acc: 90.860% (9086/1000 100/100 

Epoch: 61
 [================================================================>]  Step: 164ms | Tot: 1m30s | Loss: 0.065 | Acc: 97.840% (48920/5000 391/391 1  
 [================================================================>]  Step: 59ms | Tot: 6s78ms | Loss: 0.340 | Acc: 90.940% (9094/1000 100/100 

Epoch: 63
 [================================================================>]  Step: 159ms | Tot: 1m30s | Loss: 0.060 | Acc: 97.984% (48992/5000 391/391 1  
 [================================================================>]  Step: 56ms 